In [1]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
from google.colab import drive
import os
from google.colab import files

In [2]:
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import json
f=open('/content/drive/MyDrive/train_data.json')
g=open('/content/drive/MyDrive/test_data.json')
train_data=json.load(f)
test_data=json.load(g)

In [5]:
nlp=spacy.load("en_core_web_lg")
db=DocBin()
for text,annot in tqdm(train_data['annotations']):
  doc=nlp.make_doc(text)
  ents=[]
  for start,end,label in annot["entities"]:
    span=doc.char_span(start,end,label=label,alignment_mode="contract")
    if span is None:
      print("Entity ignored")
    else:
      ents.append(span)
  doc.ents=ents
  db.add(doc)
db.to_disk("./training_data.spacy")

100%|██████████| 20/20 [00:00<00:00, 257.64it/s]


In [6]:
nlp2=spacy.load("en_core_web_lg")
db2=DocBin()
for text,annot in tqdm(test_data['annotations']):
  doc2=nlp2.make_doc(text)
  ents2=[]
  for start,end,label in annot["entities"]:
    span2=doc2.char_span(start,end,label=label,alignment_mode="contract")
    if span2 is None:
      print("Skipping entity")
    else:
      ents2.append(span2)
  doc2.ents=ents2
  db2.add(doc2)
db2.to_disk("./test_data.spacy")

100%|██████████| 15/15 [00:00<00:00, 110.83it/s]


In [7]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
!python -m spacy train ./config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./test_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    112.50    0.00    0.00    0.00    0.00
 10     200      12266.46   8005.07   37.43   37.23   37.63    0.37
 20     400         47.81    236.59   43.08   41.18   45.16    0.43
 30     600          0.00      0.00   43.08   41.18   45.16    0.43
 40     800          0.00      0.00   45.36   43.56   47.31    0.45
 50    1000          0.00      0.00   42.49   41.00   44.09    0.42
 60    1200          0.00      0.00   42.27   40.59   44.09    0.42
 70    1400          0.00      0.00   42.27   40.59   44.09    0.42
 80    1600          0.00      0.00 

In [12]:
ner_model=spacy.load('model-best')

In [13]:
doc=ner_model("B-101 Regency apartments New Bombay 28 December 2020 Dear Vishnu I just heard that you have received a new job. I couldnt be happier for you. Congratulations! I know it is not easy to get a good job in this troubled job market. It is particularly tough for a fresh graduate with hardly any work experience. The resourcefulness you displayed in your job search is commendable. You have inspired me and a lot others that “Where there is a will, there is definitely a way.” You have made us all proud, Vishnu! I am quite confident that your knowledge of computer and artistic skills will make you a great graphics designer. I expect to hear great news coming from your direction in the coming years. May you fulfil all your dreams. Best wishes. Yours truly Sara ")


In [14]:
spacy.displacy.render(doc,style='ent',jupyter=True)